In [7]:
import tensorflow as tf

gpu = tf.config.experimental.list_physical_devices("GPU")
print(gpu)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Model 1: CIFAR10

In [8]:
from tensorflow import keras
from tensorflow.keras.layers import Flatten, Dense
import numpy as np
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train_ohc = tf.keras.utils.to_categorical(y_train, num_classes=10, dtype='float32')
y_test_ohc = tf.keras.utils.to_categorical(y_test, num_classes=10, dtype='float32')

def get_model_1():
    model = tf.keras.models.Sequential([
        Flatten(input_shape=(32, 32, 3)),
        Dense(3000, activation='relu'),
        Dense(1000, activation='relu'),
        Dense(10, activation='sigmoid')
    ])
    model.compile(optimizer="adam", 
                  loss="categorical_crossentropy", 
                  metrics=["accuracy"])    
    return model


In [13]:
# CPU vs GPU testing
def test_cpu_1(epochs=5):
    with tf.device("/CPU:0"):
        model_cpu = get_model_1()
        model_cpu.fit(x_train, y_train_ohc, epochs=epochs)

def test_gpu_1(epochs=5):
    with tf.device("/GPU:0"):
        model_gpu = get_model_1()
        model_gpu.fit(x_train, y_train_ohc, epochs=epochs)


In [16]:
# %timeit test_cpu(1)
%timeit -n1 -r1 test_cpu(epochs=1)


1563/1563 [==============================] - 58s 37ms/step - loss: 1.8880 - accuracy: 0.3275
58.7 s ± 151 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit -n1 -r1 test_gpu(epochs=10)

Epoch 1/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.8880 - accuracy: 0.3274
Epoch 2/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.6568 - accuracy: 0.4054
Epoch 3/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.5733 - accuracy: 0.4344
Epoch 4/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.5290 - accuracy: 0.4515
Epoch 5/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4912 - accuracy: 0.4657
Epoch 6/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4610 - accuracy: 0.4764
Epoch 7/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4341 - accuracy: 0.4856
Epoch 8/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4085 - accuracy: 0.4942
Epoch 9/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.3954 - accuracy: 0.5001
Epoch 10/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.374

# Model 2: Fashion MNIST

In [24]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images_scaled = train_images / 255.0
test_images_scaled = test_images / 255.0

def get_model_2(hidden_layers=1):
    # Flatten layer for input
    layers = [keras.layers.Flatten(input_shape=(28, 28))]
    # hidden layers
    for i in range(hidden_layers):
        layers.append(keras.layers.Dense(500, activation='relu'),)
    # output layer    
    layers.append(keras.layers.Dense(10, activation='sigmoid'))
    model = keras.Sequential(layers)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def test_cpu_2():
    with tf.device('/CPU:0'):
        cpu_model = get_model_2(hidden_layers=5)
        cpu_model.fit(train_images_scaled, train_labels, epochs=5)

def test_gpu_2():
    with tf.device('/GPU:0'):
        cpu_model = get_model_2(hidden_layers=5)
        cpu_model.fit(train_images_scaled, train_labels, epochs=5)

In [25]:
%timeit -n1 -r1 test_cpu_2()

Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.5200 - accuracy: 0.8133
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3909 - accuracy: 0.8606
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3521 - accuracy: 0.8721
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3267 - accuracy: 0.8807
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3084 - accuracy: 0.8876
51.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [26]:
%timeit -n1 -r1 test_gpu_2()

Epoch 1/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5161 - accuracy: 0.8138
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3896 - accuracy: 0.8600
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3544 - accuracy: 0.8721
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3269 - accuracy: 0.8819
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3086 - accuracy: 0.8884
40.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
